In [1]:
from typing import Pattern
import pandas as pd 
import numpy as np
import nltk
import csv
import re
from re import sub
import sys
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('french'))
global str


In [2]:
PATTERN_SUPP_HTML : Pattern = re.compile(r'<(.*?)>')
REPLACE_NO_SPACE : Pattern= re.compile(r"['.;:!*=\%?,<>\"()\[\]]")
REPLACE_WITH_SPACE : Pattern = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)|°")

In [3]:
# nettoyer le texte 
def preprocess_clean(reviews):
    #if type(reviews) != float:
    #print(reviews)
    reviews = str(reviews).lower() # mettre le texte en minuscule

    reviews = re.sub(r'\bagrave\b', 'a', reviews) #replacer agrave par a (!à)
    reviews = re.sub(r'\beacut\b|\begrave\b|\bpreacute\b', 'e', reviews)#
    
    sub_html = re.findall(PATTERN_SUPP_HTML, reviews) #supprimer les balise html
    for html in sub_html:
        if html:
            reviews = re.sub(r'\s+', ' ', reviews)
            reviews = re.sub(r"\'", ' ', reviews)
            html = r'<' + html + r'>'
            #reviews = re.sub(html, ' ', reviews)
            reviews = reviews.replace(html, ' ')
            reviews = reviews.replace('< >', ' ')
    
    reviews = REPLACE_NO_SPACE.sub(' ', reviews) #supprimer tout les caracteres [.;:!=\%?,\"()\[\]]
    reviews = REPLACE_WITH_SPACE.sub(' ', reviews) #supprimer espace entre balise html au cas où y'en a 
    reviews = reviews.replace('ttt', 'traitement') 
    reviews = reviews.replace('&nbsp', ' ') 
    
    reviews = re.sub(r"([+])", ' plus ', reviews)
    reviews = re.sub(r"([+])", ' ', reviews)
    reviews = re.sub(r'[0-9]', ' ', reviews)
    
    #reviews = re.sub(r'\b\w{1,3}\b',' ', reviews)

    #La première manipulation souvent effectuée dans le traitement du texte est la suppression de ce qu'on 
    #appelle en anglais les stopwords. Ce sont les mots très courants dans la langue étudiée 
    #("et", "à", "le"... en français) qui n'apportent pas de valeur informative pour la compréhension 
    #du "sens" d'un document et corpus. Il sont très fréquents et ralentissent notre travail : 
    #nous souhaitons donc les supprimer.
    for word in reviews.split() :
        if word in stop_words or len(word) < 2:
            word = r'\b' + word + r'\b'
            reviews = re.sub(word, '', reviews)
         
    reviews = re.sub(' +', ' ', reviews)#supprimer tout espace de plus qui peut exister à la fin de traitement
    #print(reviews)
    return reviews

In [22]:
output = df.drop_duplicates()
print('CHUTES : ', output.groupby('CHUTES').size())
print('prediction : ', output.groupby('prediction').size())
len(df)

CHUTES :  CHUTES
False     621
True     2670
dtype: int64
prediction :  prediction
CHUTE    3240
RIEN       51
dtype: int64


3291

In [19]:
tab_predict = []
for _, libelle2, CHUTES, prediction in df.values.tolist():
    if CHUTES == True and prediction == "RIEN":
        tab_predict.append((libelle2, CHUTES, prediction))


In [20]:
df_predict = pd.DataFrame(tab_predict, columns =['libelle2', 'CHUTES', 'prediction'])
print(len(df_predict))
df_predict.head()

51


,libelle2,CHUTES,prediction
0,AU TOUR DE NURSING DE 4H30 ÉTAIS DEBOUT PRÊTE ...,True,RIEN
1,MÉDICALCHUTE MÉCANIQUE EN CHAMBRE CET APRÈS MI...,True,RIEN
2,OBSERVATIONS : CONSULTATIONFAITES LE : 06/02/2...,True,RIEN
3,MR D ESSAI DE SE LEVER SON FAUTEUIL RISQUE ...,True,RIEN
4,OBSERVATIONS : CONSULTATIONFAITES LE : 08/02/2...,True,RIEN


In [21]:
df_predict.to_csv('data/test/data-tester-chuteXTrue-predictionXFalse.csv', sep=';', encoding='utf-8')

# Table CHUTES & SYNDROMES 

In [116]:
path_train = "data/test/exportNETSOINS_TRANSMISSIONS__semaine319.csv"
path_expo = "data/test/exportNETSOINS_CHUTES_SEMAINE319.csv"

In [117]:
from datetime import datetime

def Timestamp_to_date(temps):
     return datetime.fromtimestamp(int(temps))


In [161]:
df_train = pd.read_csv(path_train, encoding='utf-8', sep=";")
#df_train = df_train.head(10000)

#df_train.date_T = df_train['date_T'].apply(lambda x:  Timestamp_to_date(x))
df_train.libelle2 = df_train.libelle2.apply(lambda x: preprocess_clean(x))
df_train = df_train.sort_values(by=['id_resident'], ascending=True)

df_train.head()

,id_resident,id_etablissement,date_T,id_type_parametrage,libelle2,GRIPPE_IRA,GEA_DIARRHEES,SUSPICION_TIAC,CHUTES,APPEL_SAMU,VACCINATION,type_info2
9879,213,207,1581267660,2,mme vomi plus,False,False,False,False,False,False,T
26114,213,207,1581184140,2,ide prévenue,False,False,False,False,False,False,T
60950,213,207,1580918130,2,compte rendu psychologie entretienfait séance ...,False,False,False,False,False,False,T
52877,213,207,1581009225,1,mme sonnée environ voir dit quelle mal partout...,False,False,False,False,False,False,T
26115,225,207,1581008943,2,compte rendu psychologie synthèsefait séance o...,False,False,False,False,False,False,T


In [178]:
table_id = []
tmp = []
cont = 213
for id_train in df_train.values.tolist():
    if id_train[0] == cont:
        tmp.append(id_train[2])
        #print(id_train + id_expo, date_)
        #table_id.append(id_train + id_expo)
    else :
        tmp.sort(key=lambda x:x)
        for tr in tmp:
            print(Timestamp_to_date(tr))
        print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
        #table_id.append(id_train + id_expo)
        
        tmp = []
        
    cont = id_train[0]

2020-02-05 16:55:30
2020-02-06 18:13:45
2020-02-08 18:49:00
2020-02-09 18:01:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 16:43:29
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 17:11:00
2020-02-06 16:42:52
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 17:03:00
2020-02-05 19:36:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 13:41:42
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 20:16:25
2020-02-06 16:57:56
2020-02-06 17:02:52
2020-02-07 16:54:32
2020-02-07 16:54:32
2020-02-08 05:33:29
2020-02-08 18:55:03
2020-02-09 19:35:05
2020-02-09 19:35:05
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 11:18:10
2020-02-06 15:31:26
2020-02-07 10:00:23
2020-02-07 11:37:34
2020-02-09 16:58:26
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 14:26:48
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 17:47:18
2020-02-04 14:37:02
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 16:59:05
2020-02-08 09:19:51
2020

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 11:18:24
2020-02-07 11:42:41
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 16:37:02
2020-02-08 16:43:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 15:26:49
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 14:06:00
2020-02-09 16:07:02
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 16:38:08
2020-02-05 16:38:09
2020-02-06 12:24:54
2020-02-07 19:02:19
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 17:06:22
2020-02-07 11:49:21
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 11:35:14
2020-02-04 07:30:29
2020-02-04 07:30:29
2020-02-05 13:25:41
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-08 15:42:40
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 00:37:14
2020-02-05 06:30:31
2020-02-05 17:56:28
2020-02-08 05:45:17
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 05:58

2020-02-07 11:53:46
2020-02-07 11:53:46
2020-02-07 11:53:46
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 18:49:41
2020-02-09 09:39:25
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 11:08:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 17:11:00
2020-02-03 19:12:04
2020-02-07 11:57:34
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 15:54:39
2020-02-03 15:54:39
2020-02-08 16:36:32
2020-02-09 18:04:02
2020-02-09 19:22:27
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 13:52:42
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 20:12:52
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 18:44:40
2020-02-06 18:55:50
2020-02-08 09:31:26
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 19:23:00
2020-02-09 16:56:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 15:55:16
2020-02-06 17:16:15
2020-02-06 18:44:13
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 14:00:22
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 21:14:22
2020-02-04 16:14:16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 08:22:33
2020-02-07 15:08:37
$$$$

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 12:27:40
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 18:53:57
2020-02-04 18:53:58
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 14:26:14
2020-02-08 12:12:56
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-08 20:06:01
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 06:21:03
2020-02-04 06:10:00
2020-02-06 06:04:54
2020-02-08 06:10:24
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 19:04:00
2020-02-08 07:08:52
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 16:29:26
2020-02-08 14:38:39
2020-02-08 14:38:39
2020-02-09 14:20:14
2020-02-09 19:10:24
2020-02-09 19:10:24
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 22:27:16
2020-02-07 22:27:16
2020-02-08 20:08:09
2020-02-09 13:39:38
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 14:24:27
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 22:04:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 12:52:09
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 02:37:16


2020-02-05 05:24:00
2020-02-06 11:41:22
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 13:18:55
2020-02-09 03:24:00
2020-02-09 18:09:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 05:04:03
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 06:26:12
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 07:30:00
2020-02-07 10:20:17
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 10:43:22
2020-02-07 15:17:34
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-09 09:41:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 18:56:29
2020-02-05 06:07:52
2020-02-05 06:07:52
2020-02-05 18:21:02
2020-02-05 18:21:02
2020-02-06 19:30:53
2020-02-06 19:30:53
2020-02-07 07:56:37
2020-02-07 19:55:49
2020-02-07 19:55:49
2020-02-08 07:48:57
2020-02-09 20:02:28
2020-02-09 20:02:28
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 15:08:02
2020-02-07 15:08:02
2020-02-08 14:33:22
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 16:21:44
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020

...

2020-02-04 19:54:16
2020-02-04 19:54:16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 07:58:00
2020-02-03 12:28:27
2020-02-04 15:10:54
2020-02-04 19:54:16
2020-02-05 18:07:55
2020-02-07 16:43:44
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 17:04:32
2020-02-03 17:19:45
2020-02-06 16:13:13
2020-02-06 16:50:17
2020-02-07 12:27:31
2020-02-07 15:11:51
2020-02-08 16:05:29
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 11:06:52
2020-02-07 16:48:16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 09:45:34
2020-02-05 19:34:26
2020-02-07 12:25:49
2020-02-07 17:08:33
2020-02-07 20:20:36
2020-02-09 21:13:03
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 06:25:00
2020-02-06 17:26:39
2020-02-07 14:33:55
2020-02-08 20:35:48
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 16:52:25
2020-02-07 06:02:58
2020-02-07 06:36:00
2020-02-07 11:48:35
2020-02-07 15:37:43
2020-02-07 23:00:00
2020-02-08 05:15:00
2020-02-08 09:21:14
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-09 21:08:03
$$$$$$$$$$$$$$$$$$$$$$$$

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 15:02:16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 16:58:54
2020-02-04 17:39:58
2020-02-07 10:14:21
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 16:43:52
2020-02-03 19:51:11
2020-02-05 13:06:21
2020-02-05 13:06:21
2020-02-06 06:21:24
2020-02-06 06:21:24
2020-02-06 06:21:24
2020-02-06 15:24:07
2020-02-06 17:39:48
2020-02-06 18:17:42
2020-02-08 19:35:08
2020-02-08 20:29:51
2020-02-08 23:22:32
2020-02-08 23:26:29
2020-02-09 06:20:08
2020-02-09 12:45:45
2020-02-09 16:49:14
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 13:38:00
2020-02-07 10:58:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 15:14:50
2020-02-03 15:22:24
2020-02-03 16:29:19
2020-02-03 16:29:19
2020-02-03 19:10:24
2020-02-04 06:05:54
2020-02-04 06:05:54
2020-02-05 00:42:27
2020-02-05 06:15:33
2020-02-06 11:50:03
2020-02-07 11:00:58
2020-02-07 11:18:51
2020-02-07 11:18:51
2020-02-07 11:19:56
2020-02-07 11:30:31
2020-02-07 16:55

2020-02-09 16:13:29
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 15:51:08
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 11:10:11
2020-02-06 06:39:27
2020-02-08 15:05:07
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 16:11:36
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 12:56:43
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 16:46:21
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-09 15:41:33
2020-02-09 15:45:40
2020-02-09 15:45:40
2020-02-09 19:58:44
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 08:54:59
2020-02-05 11:04:03
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-06 19:55:51
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 06:02:23
2020-02-03 16:35:59
2020-02-07 14:36:05
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 05:46:18
2020-02-03 05:46:18
2020-02-07 11:25:31
2020-02-08 23:38:23
2020-02-09 11:02:15
2020-02-09 11:02:15
2020-02-09 23:35:44
$$$$$$$$$$$$$$$$$$$$

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 05:38:46
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 14:35:03
2020-02-04 11:26:04
2020-02-05 10:17:53
2020-02-08 19:08:04
2020-02-09 19:18:33
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-04 09:08:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 10:42:44
2020-02-03 16:11:33
2020-02-03 16:11:33
2020-02-03 19:24:25
2020-02-03 19:24:25
2020-02-04 14:29:11
2020-02-05 16:09:35
2020-02-05 16:10:34
2020-02-07 04:29:00
2020-02-07 04:29:00
2020-02-07 18:04:00
2020-02-07 18:38:43
2020-02-08 14:24:27
2020-02-08 14:24:27
2020-02-08 14:24:27
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-07 20:02:54
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-03 12:46:42
2020-02-06 03:13:05
2020-02-06 13:10:26
2020-02-06 13:10:26
2020-02-06 13:10:26
2020-02-07 02:30:13
2020-02-07 02:30:13
2020-02-08 17:23:00
2020-02-09 17:32:00
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2020-02-05 16:47:21
2020-02-06 05:50:21
2020-02-08 17:03:47
$$$$$$$$$$$$

KeyboardInterrupt: 

In [163]:
df_train.to_csv('data/test/marge1323-Training-data.csv', sep=';', encoding='utf-8')

In [142]:
df_expo = pd.read_csv(path_expo, encoding='latin-1', sep=",")
#df_expo.date_chute = df_expo['date_chute'].apply(lambda x:  Timestamp_to_date(x))
df_expo.analyse_cause = df_expo.analyse_cause.apply(lambda x: preprocess_clean(x))
df_expo.analyse_action = df_expo.analyse_action.apply(lambda x: preprocess_clean(x))
df_expo.autre_remarque = df_expo.autre_remarque.apply(lambda x: preprocess_clean(x))
df_expo.head()

,id_resident_chute,date_chute,hospitalisation,id_resident,id_transmission,lunette,medecin_prevenu,pieds,analyse_cause,analyse_action,...,type,aide_visuelle_utilisee,appareil_auditif,has_contention,id_ide_prevenu,id_levee_materiel,id_type_lieu,ide_prevenue,levee_tiers,mobilite
0,530964,1580684400,1,47640,20257681,1,0,2,nan,nan,...,Non concerné,1.0,0,0,NaN,0,5,0,1,1.0
1,530956,1580684400,0,158779,20257516,1,0,2,nan,nan,...,Non concerné,0.0,0,0,NaN,0,1,0,1,NaN
2,531533,1580684400,0,321420,20266433,1,0,4,nan,nan,...,De son lit avec demi-barrière,0.0,1,0,NaN,0,1,0,1,0.0
3,530917,1580685000,1,92606,20256432,0,0,2,nan,nan,...,Non concerné,NaN,0,0,NaN,0,1,0,1,1.0
4,530953,1580685000,0,270224,20257452,0,0,4,désorientée,nan,...,Non concerné,NaN,0,0,NaN,0,1,0,1,NaN


In [143]:
len(df_expo)

1663

In [144]:
len(df_expo)
heures1 = datetime.strptime("2020-02-03 08:00:00", "%Y-%m-%d %H:%M:%S")
heures2 = datetime.strptime("2020-02-03 00:00:00", "%Y-%m-%d %H:%M:%S")
heures  = heures1 - heures2
print(heures)

8:00:00


In [145]:
table_id = []

for id_train in df_train.values.tolist():
    for id_expo in df_expo.values.tolist():
        if id_train[0] == id_expo[3]:
            #print(id_train + id_expo, date_)
            table_id.append(id_train + id_expo)

In [146]:
table_id[:1]

[[2125,
  432,
  1580903023,
  2,
  "EN ATTENTE D'ANALYSE",
  False,
  False,
  False,
  False,
  False,
  False,
  'T',
  532396,
  1580884200,
  0,
  2125,
  20281129,
  0,
  0,
  4,
  'troubles cognitifs moteurs',
  'installation lit plus bas coussin demi lune poursuivre attente cales tronc lit',
  'nan',
  '["Normal"]',
  '["Aucune"]',
  '["Habillage inad\\u00e9quat","Sans cause"]',
  'Couché',
  'De son lit sans barrières',
  nan,
  0,
  1,
  2212.0,
  0,
  1,
  1,
  1,
  1.0]]

In [147]:
lissto = list(df_expo.columns.values)
#list(df_train.columns.values) + list(df_expo.columns.values)
lissto[3] = 'id_resident_2' 
lissto

['id_resident_chute',
 'date_chute',
 'hospitalisation',
 'id_resident_2',
 'id_transmission',
 'lunette',
 'medecin_prevenu',
 'pieds',
 'analyse_cause',
 'analyse_action',
 'autre_remarque',
 'cause',
 'consequence',
 'danger_json',
 'position',
 'type',
 'aide_visuelle_utilisee',
 'appareil_auditif',
 'has_contention',
 'id_ide_prevenu',
 'id_levee_materiel',
 'id_type_lieu',
 'ide_prevenue',
 'levee_tiers',
 'mobilite']

In [148]:
df_predict = pd.DataFrame(table_id, columns = list(df_train.columns.values) + lissto ) 
df_predict = df_predict.sort_values(by=['id_resident'], ascending=True)
print(len(df_predict))
#list(df_predict.columns.values)

2254


In [149]:
df_predict = df_predict[['id_resident', 'id_resident_2', 'date_T', 'date_chute', 'libelle2', 'CHUTES', 'analyse_cause', 'analyse_action', 'autre_remarque']]
df_predict.head()



,id_resident,id_resident_2,date_T,date_chute,libelle2,CHUTES,analyse_cause,analyse_action,autre_remarque
824,1636,1636,1580863661,1580862000,EN ATTENTE D'ANALYSE,False,nan,nan,dû vouloir rendre toilettes car canne trouvée...
825,1636,1636,1580863661,1580868240,EN ATTENTE D'ANALYSE,False,nan,nan,nan
0,2125,2125,1580903023,1580884200,EN ATTENTE D'ANALYSE,False,troubles cognitifs moteurs,installation lit plus bas coussin demi lune po...,nan
2236,2344,2344,1581265190,1581239700,"MME VA BIEN ELLE EST SOURIANTE ,ELLE A BIEN ...",False,changement position lit sans matelas potection,avant tout changement lit doit procéder valid...,mme glissé coté où barrière
2237,2356,2356,1581224374,1580965500,"MR M'A DEMANDÉ DU DOLIPRANE HIER SOIR ,JE LUI ...",False,nan,nan,mr tombé salle bain petite plaie joue coté droit


In [182]:
chutes = df_predict[df_predict['CHUTES'] == True]
chutes.to_csv('data/test/marge-data.csv', sep=';', encoding='utf-8')
chutes.head()

,id_resident,id_resident_2,date_T,date_chute,libelle2,CHUTES,analyse_cause,analyse_action,autre_remarque
832,6223,6223,1581054489,1581024600,RELEVÉ PAR UN TIERS : OUIRELEVÉ AVEC UN MATÉRI...,True,chute lit chambre,neuroleptiques tercian traitement hta loxen al...,nan
830,6223,6223,1581054489,1580720400,RELEVÉ PAR UN TIERS : OUIRELEVÉ AVEC UN MATÉRI...,True,nb chute analysée codir perdu équilibre chambre,neuroleptiques tercian traitement hta loxen a...,chuté chambre sans mal apparent dit avoir per...
831,6223,6223,1581054489,1581024600,RELEVÉ PAR UN TIERS : OUIRELEVÉ AVEC UN MATÉRI...,True,nan,nan,nan
836,19235,19235,1581226440,1581210000,DATE : 09/02/2020 À 02H00TÉMOIN : SANS TÉMOINL...,True,nan,nan,retrouvé sol près lit chute sans gravité appar...
2242,19401,19401,1581014718,1581013800,RELEVÉ PAR UN TIERS : NONRELEVÉ AVEC UN MATÉRI...,True,nan,nan,barrière cote fenêtre


In [186]:
tmp = []
tmp2 = []
cont = 6223
for id_train in chutes.values.tolist():
    if id_train[0] == cont:
        tmp.append(id_train[2])
        tmp2.append(id_train[3])
        #print(id_train + id_expo, date_)
        #table_id.append(id_train + id_expo)
    else :
        tmp.sort(key=lambda x:x)
        tmp2.sort(key=lambda x:x)
        for tr, tr2 in zip(tmp, tmp2):
            print(abs(Timestamp_to_date(tr) - Timestamp_to_date(tr2)))
        print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
        #table_id.append(id_train + id_expo)
        
        tmp = []
        
    cont = id_train[0]

3 days, 20:48:09
8:18:09
8:18:09
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
5 days, 9:33:29
3 days, 23:53:29
3 days, 22:33:29
1 day, 21:03:29
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2 days, 5:52:21
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
4 days, 19:39:14
3 days, 9:59:14
4 days, 12:47:57
3 days, 20:32:57
2 days, 11:17:57
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
4 days, 12:27:16
3 days, 2:47:16
3 days, 1:27:16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
5 days, 20:04:36
4 days, 10:24:36
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
2 days, 20:27:07
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
3 days, 9:23:19
$$$$$$$$$$$$$

In [49]:

for id_train in df_predict.values.tolist():
    d1 = datetime.strptime(str(id_train[1]), "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(str(id_train[4]), "%Y-%m-%d %H:%M:%S")
    if d1 > d2 : 
        print(d1 - d2)
    else :
        print(d2 - d1)

6:22:24
1:49:45
0:05:45
1:49:45
0:33:18
1:10:42
8:06:24
0:05:45
1:16:19
0:27:41
1 day, 7:42:09
1 day, 9:26:09
5 days, 1:59:27


In [31]:
from datetime import datetime

d1 = datetime.strptime("2015-08-10 19:33:27.653", "%Y-%m-%d %H:%M:%S.%f")
d2 = datetime.strptime("2015-08-10 19:31:28.209", "%Y-%m-%d %H:%M:%S.%f")

print(d1 - d2)

0:01:59.444000


In [175]:
trdd = ['1580868585', '1580747871', '1580863661', '1580863998', '1580868585', '1580891184']
trdd.sort(key=lambda x:x)
#for tr in trdd :
 #   print()
    #print(Timestamp_to_date(tr))
trdd    

['1580747871',
 '1580863661',
 '1580863998',
 '1580868585',
 '1580868585',
 '1580891184']

In [12]:
id_resident= ["144834 267105 117926 108365 105554 144373 129217 105235 117925 108028 233601 223098 288599 346619 206838 262689 123529 250220\
 201762\
 161064\
 153333\
 160464\
 259621\
 132961\
 318559\
 284425\
 161064\
 160627\
 338799\
 132550\
 209238\
 241339\
 329299\
 288978\
 158054\
 320979\
 158054\
 158054\
 285585\
 158054\
 288978\
 141461\
 162817\
 332700\
 145481\
 145185\
 146509\
 150086\
 160626\
 276243"]
id_resident22 = ["144829\
    345079\
    280041\
    225899\
    165405\
    144834\
    237160\
    289979\
    112027\
    331899\
    246118\
    267658\
    148839\
    151310\
    158777\
    158297\
    324439\
    300439\
    322498\
    161547\
    159896\
    121344\
    149789\
    221886\
    302358\
    121396\
    167179\
    157779\
    167818\
    121360\
    328998\
    121367\
    242241\
    130395\
    203682\
    204218\
    317238\
    267022\
    136736\
    136664\
    255118\
    175838\
    345178\
    329299\
    288978\
    145481\
    145465\
    263438\
    222038\
    148095\
    148015\
    301259\
    299004\
    150086\
    176072\
    188198\
    318238\
    157111\
    157053\
    347399"]

In [16]:
for sd in id_resident22[0].split():
    if sd in id_resident[0].split():
        print(sd)

144834
329299
288978
145481
150086


In [ ]:
329299	1468	KORIAN VILLA JANIN	42	1583949166	APPEL DE NORD CE JOUR POUR NOUS DIRE QUE LE RETOUR DE MR MARTIN EST PRÉVU POUR DEMAIN SOIT LE JEUDI 12/03 MAIS QU'ILS NE FERONT PAS DE DÉPISTAGE DE COVID 19. ALLO MEDEC POUR LUI DEMANDER LA CAT POUR CE RETOUR ET SI ON L'ACCEPTE OU NON. IL NOUS DIT QU'IL PEUT RENTRER DEMAIN A CONDITION QUE L'HÔPITAL NOUS FASSE UN PAPIER ÉCRIT DISANT QUE MR M. NE PRÉSENTE NI SYMPTÔMES NI FIÈVRE...LA REQUÊTE ÉTANT ACCEPTÉ PAR LA CADRE DE SANTÉ DU SERVICE, LE RETOUR DE MR M SE FERA DEMAIN DANS LA JOURNÉE.											
329299	1468	KORIAN VILLA JANIN	42	1584023980	RETOUR DE MR MARTIN CE JOUR APRÈS UNE HOSPITALISATION PROGRAMMÉE POUR UNE CHIRURGIE DE L'ÉTAGE ANTÉRIEUR DU SINUS. L'OPÉRATION S'EST DÉROULÉE SANS INCIDENTUN SCANNER À ÉTÉ RÉALISÉ LE 4 MARS QUI NE RETROUVAIT PAS DE SAIGNEMENT MAIS UNE STABILITÉ DES HÉMATOMES. REVENU AVEC UNE ORDONNANCE À JOUR, REPRENDRE LE LOVENOX 40 UNE FOIS PAR JOURPRÉCAUTIONS STANDARDS À PRENDRE CAR MR REVIENT D'UN ÉTABLISSEMENT OÙ DES CAS DE CORONAS VIRUS ÉTAIT PRÉSENT (MR N'A PAS EUX DE SYNDROME GRIPPAL PENDANT LE SÉJOUR)  ET MR À ÉTÉ DANS LE MÊME SERVICE QU'UN PATIENT PORTEUR D'UNE BHRE (DANS LES SELLES) DONC PRENDRE DES MESURES PENDANT LA TOILETTE. MR DOIT RESTÉ EN ISOLEMENT 14 JOURS !! PM DE KINÉ MARCHEA PRÉVOIR, UN  TDM CRANE SANS INJECTION POUR VOIR L'ÉVOLUTION DE L'OPÉRATION PROCHAIN RENDEZ VOUS EN CONSULTATION NEUROCHIR, LE 12/05 À 14H00											



